In [ ]:
# Google Drive Mount

from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData

/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData


In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'sudo apt autoremove' to remove them.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,671 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: fall

In [ ]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.2 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu

bash: line 6: fg: no job control


In [ ]:
%ls

2018_일자+언론사+제목+본문.csv  2020_일자+언론사+제목+본문.csv
2018_필요없는컬럼날림.csv       2020_필요없는컬럼날림.csv
2019_일자+언론사+제목+본문.csv  2021_일자+언론사+제목+본문.csv
2019_필요없는컬럼날림.csv       2021_필요없는컬럼날림.csv


In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from konlpy.tag import Okt
from pathlib import Path
import re

# 기사 토큰화

In [ ]:
def tokenizeDf(df):
  okt = Okt()
  tqdm.pandas()
  df['year'] = df['일자'].apply(lambda x: int(str(x)[:4]))
  df['month'] = df['일자'].apply(lambda x: int(str(x)[4:6]))
  df['date'] = df['일자'].apply(lambda x: int(str(x)[6:]))
  print('Preprocessing')
  df['content'] = df['본문'].progress_apply(lambda x: re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub('', x))
  tokenized_df = df[['year', 'month', 'date', '언론사', '제목', 'content']]
  # 어근 추출
  print('Tokenize')
  tokenized_df['tokenized_content'] = tokenized_df['content'].progress_apply(lambda x: okt.morphs(x, stem=True))
  tokenized_df.rename(columns = {'언론사': 'press', '제목': 'title'}, inplace = True)
  return tokenized_df

# 기사 명사 추출

In [ ]:
def nounDf(df):
  okt = Okt()
  tqdm.pandas()
  df['year'] = df['일자'].apply(lambda x: int(str(x)[:4]))
  df['month'] = df['일자'].apply(lambda x: int(str(x)[4:6]))
  df['date'] = df['일자'].apply(lambda x: int(str(x)[6:]))
  print('Preprocessing')
  df['content'] = df['본문'].progress_apply(lambda x: re.compile('[|ㄱ-ㅎ|ㅏ-ㅣ]+').sub('', x))
  noun_df = df[['year', 'month', 'date', '언론사', '제목', 'content']]
  # 명사 추출
  print('Noun')
  noun_df['noun_content'] = noun_df['content'].progress_apply(lambda x: okt.nouns(x))
  noun_df.rename(columns = {'언론사': 'press', '제목': 'title'}, inplace = True)
  return noun_df

# 전처리된 데이터프레임 저장

In [ ]:
def savePreprocessedDf(year, data_path, method = 'tokenize'):
  parent_path = os.path.abspath(os.path.join(data_path, os.pardir))
  output_path = os.path.join(parent_path, 'PreprocessedData')
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  year = str(year)
  file_name = year + '_일자+언론사+제목+본문.csv'
  file_dir = os.path.join(data_path, file_name)
  df = pd.read_csv(file_dir, encoding = 'utf-8')
  if method == 'tokenize':
    tokenized_df = tokenizeDf(df)
    # 어근 데이터프레임 csv로 저장
    tokenized_name = year + '_tokenized.csv'
    tokenized_dir = os.path.join(output_path, tokenized_name)
    tokenized_df.to_csv(tokenized_dir, encoding = 'utf-8', index = False)
  else:
    noun_df = nounDf(df)
  # 명사 데이터프레임 csv로 저장
    noun_name = year + '_noun.csv'
    noun_dir = os.path.join(output_path, noun_name)
    noun_df.to_csv(noun_dir, encoding = 'utf-8', index = False)

In [ ]:
savePreprocessedDf(2021, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'tokenize')

Preprocessing


100%|██████████| 518904/518904 [00:03<00:00, 149773.84it/s]


Tokenize


 96%|█████████▌| 498905/518904 [3:25:07<08:53, 37.49it/s]

In [ ]:
savePreprocessedDf(2021, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'noun')

In [ ]:
savePreprocessedDf(2020, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'tokenize')

In [ ]:
savePreprocessedDf(2020, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'noun')

In [ ]:
savePreprocessedDf(2019, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'tokenize')

In [ ]:
savePreprocessedDf(2019, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'noun')

In [ ]:
savePreprocessedDf(2018, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'tokenize')

In [ ]:
savePreprocessedDf(2018, '/content/gdrive/Shareddrives/컨디션난조,분위기고조/텀프로젝트/NewsPolicyRecommendation/Data/NewsData', method = 'noun')